In [4]:
from openai import OpenAI
import pandas as pd
OPENAI_API_KEY = "sk-proj-m5MPl2PEzsvzvTIgPa8jT3BlbkFJuxeFFZh3K61n7q74BinK"

client = OpenAI(api_key=OPENAI_API_KEY)



In [5]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[{'role': 'user', 'content': "say Hi"}],
  temperature=1,
  max_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [6]:
response.choices[0].message.content

'Hi! How can I assist you today?'

In [22]:
def add_line_numbers_to_string(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    numbered_lines = [f"{i}: {line}" for i, line in enumerate(lines, start=1)]
    return ''.join(numbered_lines)

class Generator:
    def __init__(self, level, vulnerability):
        self.client = OpenAI(api_key=OPENAI_API_KEY)
        
        if level=='analysor':
            self.json_formatter = "Return the response in RFC8259 compliant JSON according to the ResponseFormat schema with no other text. "
            self.message = [{"role":"system","content":"You are a cyber-security expert who analyses and audits smart contracts and provides reasoning, potential security risks nad fixed code."}]
            self.output_formatter = """[
   {"vulnerableLines": "l1-l2",
    "vulnerabilityReason": "<Reasons of the ulnerability of lines l1 to l2>",
    "potentialSecurityRisk": "<Potential risks theh vulneraility causes>",
    "fixedCode": "<the enclosing function with no bugs or vulnerability>"

  },
   {"vulnerableLines": "l3-l4",
    "vulnerabilityReason": "<Reasons of the ulnerability of lines l3 to l4>",
    "potentialSecurityRisk": "<Potential risks theh vulneraility causes>",
    "fixedCode": "<the enclosing function with no bugs or vulnerability>"
    }
  ]"""
            self.user_prefix = f"In the code below, each given lines of code (start and end) contain a {vulnerability} vulnerability. For each given lines, explain the vulnerablity reason and describe the potential security risk it introduces. Here is an example of output format\n{self.output_formatter}"
    
    
    def set_target_vulnerability(self, vulnerability):
        self.vulnerability = vulnerability
        
    def update_message(self, new_message):
        self.message.append(new_message)
        
    def get_user_message(self, instruction, code, lines_of_code):
        self.user_content = f"""
{self.user_prefix}

Instruction:

{instruction}

Smart Contract Code:
{code}

Lines of codes:
{lines_of_code}

---
{self.json_formatter}
###
        """
        self.user_message = {"role": "user", "content":self.user_content}

    def create_prompt(self, instruction, code, lines_of_code):
        self.get_user_message(instruction, code, lines_of_code)
        self.message.append(self.user_message)

    def generate(self):
        
        completion = self.client.chat.completions.create(
          model="gpt-4o-mini",
          messages = self.message,
          temperature=1,
          max_tokens=3200,
          top_p=1.,
          frequency_penalty=0,
          presence_penalty=0,
          stop=None
        )
        answer = completion.choices[0].message.content
        return answer, completion

In [47]:
data_loc = "../../Dataset/IoU/SolidiFi_IoU/draft"
os.listdir(data_loc)
file_name = "renamed_no-commented_buggy_1"
sol_file_name = "renamed_no-commented_buggy_1.sol"
code = add_line_numbers_to_string(os.path.join(data_loc, sol_file_name))


csv_file_name = "no-commented_BugLog_1.csv"
df = pd.read_csv(os.path.join(data_loc, csv_file_name)).sort_values(by='loc')
result = df.apply(lambda row: f"{row['loc']} - {row['loc'] + row['length']-1}", axis=1)
lines_of_code = '\n'.join(result)

instruction_name = "../../Dataset/temp_dataset/clean_dataset/IoU/instruction.txt"
with open(instruction_name, 'r') as file:
    instruction = file.read()



In [27]:
data_loc = "../../Dataset/RE/SolidiFi_RE/draft"
os.listdir(data_loc)
file_name = "renamed_no-commented_buggy_1"
sol_file_name = "renamed_no-commented_buggy_1.sol"
# with open(os.path.join(data_loc, sol_file_name), 'r') as file:
#     code = file.read()
code = add_line_numbers_to_string(os.path.join(data_loc, sol_file_name))


csv_file_name = "no-commented_BugLog_1.csv"
df = pd.read_csv(os.path.join(data_loc, csv_file_name)).sort_values(by='loc')
result = df.apply(lambda row: f"{row['loc']} - {row['loc'] + row['length']-1}", axis=1)
lines_of_code = '\n'.join(result)

instruction_name = "../../Dataset/temp_dataset/clean_dataset/RE/instruction.txt"
with open(instruction_name, 'r') as file:
    instruction = file.read()

In [57]:
data_loc = "../../Dataset/TD/SolidiFi_TD/draft"
os.listdir(data_loc)
file_name = "renamed_no-commented_buggy_1"
sol_file_name = "renamed_no-commented_buggy_1.sol"
code = add_line_numbers_to_string(os.path.join(data_loc, sol_file_name))


csv_file_name = "no-commented_BugLog_1.csv"
df = pd.read_csv(os.path.join(data_loc, csv_file_name)).sort_values(by='loc')
result = df.apply(lambda row: f"{row['loc']} - {row['loc'] + row['length']-1}", axis=1)
lines_of_code = '\n'.join(result)

instruction_name = "../../Dataset/temp_dataset/clean_dataset/TD/instruction.txt"
with open(instruction_name, 'r') as file:
    instruction = file.read()

In [58]:
print(lines_of_code)


6 - 10
13 - 21
24 - 28
31 - 39
42 - 44
46 - 46
48 - 48
54 - 58
60 - 64
66 - 69
71 - 74
76 - 84
94 - 98
107 - 111
124 - 126
131 - 131
138 - 138
143 - 143


In [59]:
generator = Generator("analysor", "Integer Overflow/Underflow")

In [60]:
generator.create_prompt(instruction, code, lines_of_code)

In [61]:
response, _ = generator.generate()

In [104]:
with open(os.path.join(data_loc, f'vulnerabilities-{file_name}.json'), 'w') as f:
    json.dump(json.loads(response[8:-3]), f, indent=2)


In [64]:
# print(code)

In [44]:
a = json.loads(response[8:-3])
for f in a:
    # print(a["fixedCode"])
    print(f["vulnerableLines"])
    print(f['fixedCode'].replace(";", ";\n"))

    print(5*"\n")

6-13
function callFunctionWithLimit() public { require(counterCallLimit <= 5);
 counterCallLimit += 1;
 if (!(msg.sender.send(10 ether))) { revert();
 } }






16-23
function buyTicket() public { require(!(lastPlayerAddress.send(jackpotAmount)));
 lastPlayerAddress = msg.sender;
 jackpotAmount = address(this).balance;
 }






26-33
function callFunctionWithLimitTwo() public { require(counterCallLimitTwo <= 5);
 counterCallLimitTwo += 1;
 if (!(msg.sender.send(10 ether))) { revert();
 } }






36-43
function buyTicketTwo() public { require(!(lastPlayerAddressTwo.send(jackpotAmountTwo)));
 lastPlayerAddressTwo = msg.sender;
 jackpotAmountTwo = address(this).balance;
 }






46-51
function withdrawBalances() public { uint256 balance = balancesMappingOne[msg.sender];
 (bool success,) = msg.sender.call.value(balance)('');
 if (success) balancesMappingOne[msg.sender] = 0;
 }






53-58
function withdrawFunds(uint256 _weiToWithdraw) public { require(balancesMappingTwo[msg.sender] >= _wei

In [63]:
print(response[8:-3])

[
   {
      "vulnerableLines": "6-10",
      "vulnerabilityReason": "The function playTimeStamp39 uses block.timestamp to compare against a calculated future time. This can lead to exploiters manipulating the block timestamp to gain an unintended advantage.",
      "potentialSecurityRisk": "A miner could set the timestamp to match the winning condition, allowing them or a colluding party to win instantly, leading to potential loss of funds for other players.",
      "fixedCode": "function playTimeStamp39(uint startTime) public { require(startTime + (5 * 1 days) <= block.timestamp, 'Game not started or already finished'); winnerTimeStamp39 = msg.sender; }"
   },
   {
      "vulnerableLines": "13-21",
      "vulnerabilityReason": "Similar to other instances, the function paymentIssue36 uses block.timestamp in a manner that allows indirectly controlling the flow of funds based on a timestamp controlled by miners.",
      "potentialSecurityRisk": "By manipulating block.timestamp, an attac